In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from utils import (
    obtener_escalar_de_inflacion,
    show_plot,
)

In [ ]:
def set_bias(last_check: datetime) -> int:
    current_date = datetime.now()
    years_diff = current_date.year - last_check.year
    months_diff = current_date.month - last_check.month
    return years_diff * 12 + months_diff

In [ ]:
last_check = datetime(
    2025, 2, 1
)  # ACTUALIZAR CUANDO SE AGREGUEN DATOS FUTUROS A ESTA FECHA Y REVISAR TODOS LOS DATOS
MONTHS_TO_UPDATE = set_bias(last_check)
MONTHS_TO_UPDATE

In [ ]:
datos_personales = pd.read_csv("../data/sueldos.csv")

In [ ]:
datos_personales["sueldo_bruto_usd_oficial"] = (
    datos_personales["sueldo_bruto_ars"] / datos_personales["dolar_oficial"]
)
datos_personales["sueldo_bruto_usd_blue"] = (
    datos_personales["sueldo_bruto_ars"] / datos_personales["dolar_blue"]
)
datos_personales["sueldo_neto_usd_oficial"] = (
    datos_personales["sueldo_neto_ars"] / datos_personales["dolar_oficial"]
)
datos_personales["sueldo_neto_usd_blue"] = (
    datos_personales["sueldo_neto_ars"] / datos_personales["dolar_blue"]
)
datos_personales.tail()

In [ ]:
rem_bruta_sec_priv = pd.read_csv(
    "https://infra.datos.gob.ar/catalog/sspm/dataset/153/distribution/153.1/download/remuneracion-bruta-asalariados-registrados-sector-privado-mensual-pesos-base-2009.csv"
)
rem_bruta_sec_priv.tail(3)

In [ ]:
datos_arg: list[dict[str, any]] = []
datos_arg.append(
    {
        "fecha": rem_bruta_sec_priv.iloc[-1]["indice_tiempo"],
        "valor": float(
            rem_bruta_sec_priv.iloc[-1]["asalariados_priv_remuneracion_mediana"]
        )
        * obtener_escalar_de_inflacion(MONTHS_TO_UPDATE-1),
        "label": "Bruto Mediana",
    }
)
datos_arg.append(
    {
        "fecha": rem_bruta_sec_priv.iloc[-1]["indice_tiempo"],
        "valor": float(
            rem_bruta_sec_priv.iloc[-1]["asalariados_priv_remuneracion_promedio"]
        )
        * obtener_escalar_de_inflacion(MONTHS_TO_UPDATE-1),
        "label": "Bruto Promedio",
    }
)

In [ ]:
distribucion_personal_ingreso_por_deciles_trismestrales = pd.read_csv(
    "https://infra.datos.gob.ar/catalog/sspm/dataset/301/distribution/301.1/download/distribucion-personal-ingreso-por-deciles-trismestrales.csv"
)
distribucion_personal_ingreso_por_deciles_trismestrales.tail(3)

In [ ]:
res = distribucion_personal_ingreso_por_deciles_trismestrales.iloc[-1][
    "decil_8_oip":"decil_10_oip"
]
datos_arg.extend(
    [
        {
            "fecha": distribucion_personal_ingreso_por_deciles_trismestrales[
                "indice_tiempo"
            ].iloc[-1],
            "valor": float(value) * obtener_escalar_de_inflacion(MONTHS_TO_UPDATE+4),
            "label": index,
        }
        for index, value in res.items()
    ]
)

In [ ]:
remuneracion_imponible_promedio_trabajadores_estables_ripte = pd.read_csv(
    "https://infra.datos.gob.ar/catalog/sspm/dataset/158/distribution/158.1/download/remuneracion-imponible-promedio-trabajadores-estables-ripte-total-pais-pesos-serie-mensual.csv"
)
remuneracion_imponible_promedio_trabajadores_estables_ripte.tail(3)

In [ ]:
datos_arg.append(
    {
        "fecha": remuneracion_imponible_promedio_trabajadores_estables_ripte.iloc[-1][
            "indice_tiempo"
        ],
        "valor": float(
            remuneracion_imponible_promedio_trabajadores_estables_ripte.iloc[-1][
                "ripte"
            ]
        )
        * obtener_escalar_de_inflacion(MONTHS_TO_UPDATE-2),
        "label": "RIPTE",
    }
)

In [ ]:
datos_arg.append(
    {
        "fecha": str(datos_personales["año"].iloc[-1])
        + "-"
        + str(datos_personales["mes"].iloc[-1])
        + "-01",
        "valor": float(datos_personales["sueldo_bruto_ars"].iloc[-1]),
        "label": "Mi Bruto",
    }
)
datos_arg.append(
    {
        "fecha": str(datos_personales["año"].iloc[-1])
        + "-"
        + str(datos_personales["mes"].iloc[-1])
        + "-01",
        "valor": float(datos_personales["sueldo_neto_ars"].iloc[-1]),
        "label": "Mi Neto",
    }
)
datos_arg.sort(key=lambda x: x["valor"])

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

bars = ax.bar(
    [dato["label"] for dato in datos_arg],
    [dato["valor"] for dato in datos_arg],
    width=0.9,
)

for bar in bars:
    height = bar.get_height()
    ax.text(
        bar.get_x() + bar.get_width() / 2,
        height,
        f"${height:,.0f}",
        ha="center",
        va="bottom",
    )

show_plot(
    fig,
    "Categorías",
    "Sueldo en Millones",
    "Comparación del Sueldo Actual con Datos del Gobierno",
)